In [ ]:
# x.stride() 是一个方法，用于获取张量 x 的每个维度的步幅（stride）。
# 步幅定义了在多维数组中，从一个元素移动到下一个元素的内存偏移量。

In [392]:
import torch

N, C, H, W = 10, 3, 32, 32
x = torch.empty(N, C, H, W)
print(x.stride())  # Outputs: (3072, 1024, 32, 1)
x = x.to(memory_format=torch.channels_last)
print(x.shape)  # Outputs: (10, 3, 32, 32) as dimensions order preserved
print(x.stride())  # Outputs: (3072, 1, 96, 3)
print(x.is_contiguous())
x = x.to(memory_format=torch.contiguous_format)
print(x.stride())  # Outputs: (3072, 1024, 32, 1)
print(x.is_contiguous())

(3072, 1024, 32, 1)
torch.Size([10, 3, 32, 32])
(3072, 1, 96, 3)
False
(3072, 1024, 32, 1)
True


In [397]:
# Alternative option
x = x.contiguous(memory_format=torch.channels_last)
print(x.stride())  # Outputs: (3072, 1, 96, 3)
print(x.is_contiguous())
print(x.is_contiguous(memory_format=torch.channels_last))  # Outputs: True


(3072, 1, 96, 3)
False
True


In [394]:

# 在 PyTorch 中，x.contiguous() 方法用于返回一个在内存中是连续的张量。理解这个概念对于优化内存访问和性能非常重要。以下是关于 x.contiguous() 的详细解释。

# 1. 什么是连续的张量？
# 在 PyTorch 中，张量的内存布局可以是连续的或非连续的。一个连续的张量表示其数据在内存中是连续存储的，这意味着可以一次性读取整个张量的数据。而非连续的张量则可能由于切片、转置或其他操作而导致数据在内存中不再是连续的。

# 2. 为什么需要 contiguous()？
# 某些操作和函数（例如，某些类型的视图操作、重塑操作等）要求输入张量是连续的。如果输入张量不是连续的，您可能会遇到错误或性能下降。在这种情况下，您可以使用 x.contiguous() 来确保张量是连续的。

# 3. 使用 x.contiguous()
# 当您调用 x.contiguous() 时，它会检查张量的内存布局。如果张量是连续的，它将返回原始张量；如果不是，它将创建一个新的连续张量，并将原始数据复制到这个新张量中。

In [389]:
# 建议用.to 而不是 .contiguous
# 在 C=1 或 (H=1且W=1) 的情况下，tensor的连续性是模糊的
# 这时候，调用 contiguous 成为一个无操作（no-op），并不会更新步幅（stride）。相反，to 方法会在维度大小为 1 的情况下，对张量进行重新步幅调整，以正确表示所需的内存格式
special_x = torch.empty(4, 1, 4, 4)
print(special_x.is_contiguous(memory_format=torch.channels_last))  # Outputs: True
print(special_x.is_contiguous(memory_format=torch.contiguous_format))  # Outputs: True

True
True


In [398]:
# 同样的事情也适用于显式置换 API permute。在可能出现歧义的特殊情况下，permute不能保证产生正确携带预期内存格式的步幅。我们建议使用to显式内存格式以避免意外行为。

# 需要注意的是，在C==1 && H==1 && W==1极端情况下，当三个非批量维度都等于1（）时，当前实现无法将张量标记为通道最后的记忆格式。

In [399]:
x = torch.empty(N, C, H, W, memory_format=torch.channels_last)
print(x.stride())  # Outputs: (3072, 1, 96, 3)
# clone保留内存格式
y = x.clone()
print(y.stride())  # Outputs: (3072, 1, 96, 3)
# to, cuda, float… 保留内存格式
if torch.cuda.is_available():
    y = x.cuda()
    print(y.stride())  # Outputs: (3072, 1, 96, 3)
# empty_like,*_like运算符保留内存格式
y = torch.empty_like(x)
print(y.stride())  # Outputs: (3072, 1, 96, 3)
# 逐点运算符保留内存格式
z = x + y
print(z.stride())  # Outputs: (3072, 1, 96, 3)

(3072, 1, 96, 3)
(3072, 1, 96, 3)
(3072, 1, 96, 3)
(3072, 1, 96, 3)
(3072, 1, 96, 3)


In [400]:
# Conv，Batchnorm 使用支持通道last的cudnn backends最后（仅适用于cudnn >= 7.6）。卷积模块，不像二进制p-wise运算符，将通道last作为主要的内存格式。如果所有输入都是连续存储器格式，则运算符产生连续存储器格式的输出。否则，输出将采用通道的最后内存格式
if torch.backends.cudnn.is_available() and torch.backends.cudnn.version() >= 7603:
    model = torch.nn.Conv2d(8, 4, 3).cuda().half()
    model = model.to(memory_format=torch.channels_last)  # Module parameters need to be channels last

    input = torch.randint(1, 10, (2, 8, 4, 4), dtype=torch.float32, requires_grad=True)
    input = input.to(device="cuda", memory_format=torch.channels_last, dtype=torch.float16)

    out = model(input)
    print(out.is_contiguous(memory_format=torch.channels_last))  # Outputs: True

True
